![Sami](https://scontent.fdac178-1.fna.fbcdn.net/v/t39.30808-6/398173458_212185451895611_2740129820745384762_n.jpg?_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_eui2=AeG-JZ_D-TuX1tPLp6RRtYv76bBYGzSjgNvpsFgbNKOA27wm9UCdQHpc_FWroK0-3PD3u1xM032yK-MjdjjqAsRs&_nc_ohc=chEyTwOPCnkQ7kNvgF-r4ce&_nc_ht=scontent.fdac178-1.fna&oh=00_AYD8r4XjEk4rJKIpff6jbZvgJWCqsUhDhIJg8QkVzcaiaQ&oe=6679E45D)

**This is tesing Image for captioning**

In [1]:
image_url = 'https://scontent.fdac178-1.fna.fbcdn.net/v/t39.30808-6/398173458_212185451895611_2740129820745384762_n.jpg?_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_eui2=AeG-JZ_D-TuX1tPLp6RRtYv76bBYGzSjgNvpsFgbNKOA27wm9UCdQHpc_FWroK0-3PD3u1xM032yK-MjdjjqAsRs&_nc_ohc=chEyTwOPCnkQ7kNvgF-r4ce&_nc_ht=scontent.fdac178-1.fna&oh=00_AYD8r4XjEk4rJKIpff6jbZvgJWCqsUhDhIJg8QkVzcaiaQ&oe=6679E45D'

# ***GPT-2 Image Captioning***

In [2]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
import requests
from io import BytesIO

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_sources):
    images = []
    for image_source in image_sources:
        if image_source.startswith('http://') or image_source.startswith('https://'):
            response = requests.get(image_source)
            i_image = Image.open(BytesIO(response.content))
        else:
            i_image = Image.open(image_source)
        
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")

        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

predictions = predict_step([image_url])
print(predictions)

2024-06-20 13:25:22.309465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 13:25:22.309529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 13:25:22.311150: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

['a man in a suit and tie standing in a field']


# ***BLIP Image Captioning (Base)***

In [3]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

img_url = image_url
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print("Caption:",processor.decode(out[0], skip_special_tokens=True))

Caption: a man standing in front of a tree


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# ***BLIP Image Captioning (Large)***

In [4]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

img_url = image_url
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print("Caption:",processor.decode(out[0], skip_special_tokens=True))

Caption: there is a man standing in a park with a neck tie


# ***An Image Describer - XTuner: A Toolkit for Efficiently Fine-tuning LLM***

In [5]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "xtuner/llava-phi-3-mini-hf"
prompt = "<|user|>\n<image>\nWhat are these?<|end|>\n<|assistant|>\n"
image_file = image_url

torch.cuda.empty_cache()
torch.backends.cuda.max_split_size_mb = 32

model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_id)
response = requests.get(image_url, stream=True)
raw_image = Image.open(response.raw)

inputs = processor(prompt, raw_image, return_tensors='pt').to(torch.device("cuda" if torch.cuda.is_available() else "cpu"), torch.float16)

output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



What are these? The image features a man standing in a park, surrounded by trees. He is wearing a blue shirt, glasses, and a lanyard around his neck. The lanyard has a badge attached to it, which is visible in the image. The man appears to be enjoying his time in the park, possibly attending an event or gathering. The trees in the background provide a natural and serene setting for the scene.
